# From photo to piece


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from snap_fit.config.snap_fit_config import get_snap_fit_paths
from snap_fit.image.process import find_contours, find_white_regions
from snap_fit.image.utils import (
    cut_rect_from_image,
    display_image,
    draw_regions,
    flip_colors_bw,
    load_image,
    pad_rect,
    save_image,
    show_image_mpl,
    sort_rects,
)
from snap_fit.puzzle.sheet import Sheet

In [ ]:
sf_paths = get_snap_fit_paths()
data_fol = sf_paths.data_fol
sample_fol = data_fol / "sample"

In [ ]:
# img_fn = "PXL_20241130_105107220.jpg"
# img_fn = "front_01.jpg"
# img_fn = "back_proc_02.jpg"
img_fn = "back_02.jpg"
# img_fn = "puzzle_pieces_01.jpeg"
img_fp = sample_fol / img_fn
img_fp

In [ ]:
sheet = Sheet(img_fp)

In [ ]:
show_image_mpl(sheet.img_orig)

In [ ]:
show_image_mpl(sheet.img_bw)

In [ ]:
for i, (x, y, w, h) in enumerate(sheet.regions[:10]):
    print(
        f"Black Region {i + 1}: x={x}, y={y}, width={w}, height={h}, area={w*h/1000:.2f}k"
    )

In [ ]:
annotated_image = draw_regions(sheet.img_orig, sheet.regions[:10])
show_image_mpl(annotated_image)

In [ ]:
for i, contour in enumerate(sheet.contours[:1]):
    print(f"Contour {i + 1}: {contour}")

## grab a chunk of image


In [ ]:
region = sheet.regions[0]
region

In [ ]:
region_pad = pad_rect(region, 30, sheet.img_bw)
region_pad

In [ ]:
annotated_image = draw_regions(annotated_image, [region_pad])
show_image_mpl(annotated_image)

#### Use the Piece class


In [ ]:
piece = sheet.pieces[0]

In [ ]:
show_image_mpl(piece.img_bw)

In [ ]:
show_image_mpl(piece.img_orig)

### Find corners in the piece


In [ ]:
from snap_fit.image.process import find_corners
from snap_fit.image.utils import draw_corners

In [ ]:
corners = find_corners(
    piece.img_bw,
    max_corners=10,
)
corners

In [ ]:
img_corners = draw_corners(piece.img_orig, corners)
show_image_mpl(img_corners)

#### Use SIFT and SURF


In [ ]:
# from snap_fit.image.process import find_surf_keypoints
from snap_fit.image.process import find_sift_keypoints
from snap_fit.image.utils import draw_keypoints

In [ ]:
# keypoints, descriptors = find_surf_keypoints(piece.img_bw)
keypoints, descriptors = find_sift_keypoints(piece.img_bw)

In [ ]:
img_keypoints = draw_keypoints(piece.img_orig, keypoints)
show_image_mpl(img_keypoints)